# Step 1: Install Dependencies

In [ ]:
!pip install fastapi uvicorn pyngrok python-multipart nest_asyncio requests sentence-transformers pytesseract pdf2image

# Step 2: Configure Ngrok
Paste your token below.

In [ ]:
from pyngrok import ngrok

# PASTE YOUR TOKEN BELOW
NGROK_TOKEN = "YOUR_NGROK_TOKEN_HERE"

ngrok.set_auth_token(NGROK_TOKEN)

# Step 3: Run the Librarian Node
If you get an error 'endpoint already online', try running this cell again.

In [ ]:
# === JURIS-CAPE WORKER NODE A: THE LIBRARIAN ===

import uvicorn
import asyncio
import os
import time
from fastapi import FastAPI, Request, HTTPException
from fastapi.responses import JSONResponse
from pyngrok import ngrok
from pydantic import BaseModel

# --- CONFIGURATION ---
SWARM_SECRET = "change-me-in-prod-secure-swarm-key"
ROLE = "librarian"

app = FastAPI()

@app.middleware("http")
async def verify_secret(request: Request, call_next):
    if request.url.path in ["/docs", "/openapi.json", "/favicon.ico"]:
        return await call_next(request)
    
    secret = request.headers.get("X-Swarm-Secret")
    if secret != SWARM_SECRET:
        return JSONResponse(status_code=403, content={"detail": "Unauthorized Node Access"})
    return await call_next(request)

# --- MODELS ---
class OCRRequest(BaseModel):
    file_url: str
    
class EmbedRequest(BaseModel):
    text: str

# --- ENDPOINTS ---
@app.post("/ocr")
async def perform_ocr(payload: OCRRequest):
    print(f"Processing OCR for {payload.file_url}")
    return {"text": "[MOCK] OCR Result for " + payload.file_url}

@app.post("/embed")
async def perform_embedding(payload: EmbedRequest):
    return {"vector": [0.0]*768}

# --- RUNNER ---
print("Cleaning up old tunnels...")

# 1. Python Kill
ngrok.kill()

# 2. System Kill (Linux force)
os.system("pkill ngrok")
time.sleep(1) # Give it a moment to die

try:
    ngrok_tunnel = ngrok.connect(8000)
    print("\n=== PUBLIC URL ===")
    print(ngrok_tunnel.public_url)
    print("====================\n")

    config = uvicorn.Config(app, port=8000)
    server = uvicorn.Server(config)
    await server.serve()
except Exception as e:
    print(f"\nFATAL ERROR: {e}")
    print("Try: Runtime > Disconnect and Delete Runtime")
